# Importa as bibliotecas da aplicação

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=52994019d3124774ea6670620c55d7dfcd07c146f8975d3ce50aa70b1ffc03ac
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import sys

from IPython.display import display
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from IPython.display import HTML, display
import tabulate

import pandas as pd
import numpy as np
import re

pd.options.display.max_colwidth = 100

# Configura o ambiente de execução do Apache Spark

In [3]:
spark = SparkSession \
        .builder \
        .master("local[2]") \
        .appName("Netflix Analysis") \
        .config("spark.executor.cores", "2") \
        .config("spark.executor.memory", "2g") \
        .config("spark.cores.max", "4") \
        .getOrCreate()

# Realiza a leituras dos dados

In [4]:
!wget https://datasets.imdbws.com/title.akas.tsv.gz

--2022-03-06 17:15:09--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 65.8.249.127, 65.8.249.4, 65.8.249.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|65.8.249.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265207918 (253M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 252.92M  52.7MB/s    in 5.0s    

2022-03-06 17:15:15 (50.6 MB/s) - ‘title.akas.tsv.gz’ saved [265207918/265207918]



In [5]:
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-03-06 17:15:15--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 18.64.174.31, 18.64.174.128, 18.64.174.10, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|18.64.174.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6091797 (5.8M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz’

title.ratings.tsv.g 100%[===================>]   5.81M  10.2MB/s    in 0.6s    

2022-03-06 17:15:15 (10.2 MB/s) - ‘title.ratings.tsv.gz’ saved [6091797/6091797]



In [6]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz

--2022-03-06 17:15:16--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 18.64.174.31, 18.64.174.128, 18.64.174.10, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|18.64.174.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153141258 (146M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 146.05M  47.3MB/s    in 3.2s    

2022-03-06 17:15:19 (45.2 MB/s) - ‘title.basics.tsv.gz’ saved [153141258/153141258]



In [7]:
!wget https://datasets.imdbws.com/title.episode.tsv.gz

--2022-03-06 17:15:19--  https://datasets.imdbws.com/title.episode.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 18.64.174.31, 18.64.174.128, 18.64.174.10, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|18.64.174.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35518152 (34M) [binary/octet-stream]
Saving to: ‘title.episode.tsv.gz’

title.episode.tsv.g 100%[===================>]  33.87M  53.5MB/s    in 0.6s    

2022-03-06 17:15:20 (53.5 MB/s) - ‘title.episode.tsv.gz’ saved [35518152/35518152]



In [8]:
!gzip -d title.akas.tsv.gz

In [9]:
!gzip -d title.ratings.tsv.gz

In [10]:
!gzip -d title.basics.tsv.gz

In [11]:
!gzip -d title.episode.tsv.gz

In [12]:
!ls

sample_data	title.basics.tsv   title.ratings.tsv
title.akas.tsv	title.episode.tsv


In [13]:
# IMDb Title Akas DataFrame
akas_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.akas.tsv")

In [14]:
# IMDb Title Ratings DataFrame
ratings_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.ratings.tsv")

In [15]:
# IMDb Title Basics DataFrame
basics_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.basics.tsv")

In [16]:
# IMDb Title Episode DataFrame
episode_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.episode.tsv")

In [17]:
# Netflix Streaming History DataFrame
netflix_df = pd.read_csv('https://raw.githubusercontent.com/vichShir/netflix-analysis/master/data/NetflixViewingHistory2022.csv')

# Confere a estrutura dos dados

In [18]:
print(type(akas_df))
print(type(ratings_df))
print(type(basics_df))
print(type(episode_df))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [19]:
akas_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)



In [20]:
ratings_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [21]:
basics_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [22]:
episode_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: string (nullable = true)
 |-- episodeNumber: string (nullable = true)



In [23]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   450 non-null    object
 1   Date    450 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


In [24]:
sum(netflix_df.duplicated())

0

# Pré-visualização dos dados

In [25]:
# IMDb Title Akas DataFrame
display(akas_df.limit(5).toPandas())

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [26]:
# IMDb Title Ratings DataFrame
display(ratings_df.limit(5).toPandas())

,tconst,averageRating,numVotes
0,tt0000001,5.7,1864
1,tt0000002,6.0,244
2,tt0000003,6.5,1632
3,tt0000004,5.8,158
4,tt0000005,6.2,2459


In [27]:
# IMDb Title Basics DataFrame
display(basics_df.limit(5).toPandas())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [28]:
# IMDb Title Episode DataFrame
display(episode_df.limit(5).toPandas())

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0020666,tt15180956,1,2
1,tt0020829,tt15180956,1,1
2,tt0021166,tt15180956,1,3
3,tt0021612,tt15180956,2,2
4,tt0021655,tt15180956,2,5


In [29]:
# Netflix Streaming History DataFrame
netflix_df.head()

,Title,Date
0,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação da Itália,14/02/2022
1,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha de Kursk,08/02/2022
2,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação do Norte da África,31/01/2022
3,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha do Atlântico,30/12/2021
4,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Dunquerque,27/12/2021


# Separar o título e o nome do episódio

In [18]:
netflix_df['Title']

0               Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação da Itália
1                 Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha de Kursk
2      Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação do Norte da África
3             Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha do Atlântico
4                       Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Dunquerque
                                                 ...                                            
445                                                                                   ParaNorman
446                                                           Bob Esponja - Um Herói Fora D'Água
447                                                             Pets - A Vida Secreta dos Bichos
448                                                The Walking Dead: Temporada 1: Adeus, passado
449                           

In [19]:
# Remove duplicated row error
netflix_df = netflix_df[netflix_df['Title'] != 'Mars: Season 1_hook_04_16x9']

In [20]:
def split_title(title):
  """
    Return the main title and episode name by title
  """
  main_title = title
  episode_name = ''
  texts = re.split('Temporada |Minissérie', title)

  # title have the separator key
  if len(texts) > 1:
    # get the main title
    main_title = texts[0]
    # remove white spaces
    main_title = main_title.strip()
    # remove the ':' character
    main_title = main_title[:-1]

    # get the episode name
    episode_name = texts[1]

  return (main_title, episode_name)

In [21]:
# Get episode name from Title
netflix_df['Episode'] = netflix_df['Title'].map(lambda x: split_title(x)[1])

In [22]:
# Update Title to the main title
netflix_df['Title'] = netflix_df['Title'].map(lambda x: split_title(x)[0])

In [23]:
netflix_df.head()

,Title,Date,Episode
0,Segunda Guerra em Cores: Caminho para a Vitória,14/02/2022,1: Ocupação da Itália
1,Segunda Guerra em Cores: Caminho para a Vitória,08/02/2022,1: Batalha de Kursk
2,Segunda Guerra em Cores: Caminho para a Vitória,31/01/2022,1: Ocupação do Norte da África
3,Segunda Guerra em Cores: Caminho para a Vitória,30/12/2021,1: Batalha do Atlântico
4,Segunda Guerra em Cores: Caminho para a Vitória,27/12/2021,1: Dunquerque


In [24]:
netflix_df.sample(10)

,Title,Date,Episode
141,O Universo,20/05/2020,1: Espaçonave Terra
323,Um maluco no pedaço,30/10/2019,4: Batizado classe A
206,Três é Demais,08/01/2020,3: O retorno de velhos hábitos
295,Um maluco no pedaço,12/11/2019,6: Bem passado
434,One-Punch Man,11/09/2019,1: Ciclista da Justiça
281,Um maluco no pedaço,17/11/2019,6: Essa foi por pouco
296,Um maluco no pedaço,12/11/2019,5: Para quem os sinos tocam
235,Três é Demais,26/12/2019,2: Promoção dos Beach Boys
332,Um maluco no pedaço,26/10/2019,4: A vitória vem com esforço
165,Três é Demais,05/02/2020,4: Banho de espuma para peixe


# Associar um ID do IMDb

In [25]:
titles = netflix_df['Title'].unique()
titles = np.sort(titles)
titles

array(['A Garota que conquistou o tempo', 'A Viagem de Chihiro',
       'A Voz do Silêncio – Koe No Katachi', 'Ano Hana',
       'Atlantis - O reino perdido', 'Bob Esponja',
       "Bob Esponja - Um Herói Fora D'Água", 'Challenger: Voo Final',
       'Coraline e o mundo secreto', 'Crianças Lobo', 'Divertida Mente',
       'Escola de rock', 'Explicando... O Coronavírus',
       'Exterminadores do Além Contra a Loira do Banheiro',
       'Grandes Momentos da Segunda Guerra em Cores',
       'História: Direto ao Assunto', 'Kubo e as Cordas Mágicas',
       'Little Witch Academia',
       'Little Witch Academia: The Enchanted Parade',
       'Luzes no Céu: Fireworks', 'Marte', 'Meu Amigo Totoro',
       'Mission Control: The Unsung Heroes of Apollo',
       'NOVA: Black Hole Apocalypse', 'NOVA: Viagem a Saturno',
       'O Universo', 'Olhos de Gato', 'One-Punch Man', 'ParaNorman',
       'Pets - A Vida Secreta dos Bichos', 'Sangatsu no Lion',
       'Segunda Guerra em Cores: Caminho para a

In [26]:
print(f'São ao todo {len(titles)} títulos.')

São ao todo 44 títulos.


In [27]:
original_titles = np.array(['Toki o kakeru shôjo', 'Sen to Chihiro no kamikakushi', 'Koe no katachi', 'Ano hi mita hana no namae o bokutachi wa mada shiranai.',
                            'Atlantis: The Lost Empire', 'SpongeBob SquarePants', 'The SpongeBob Movie: Sponge Out of Water', 'Challenger: The Final Flight',
                            'Coraline', 'Ôkami kodomo no Ame to Yuki', 'Inside Out', 'School of Rock',
                            'Coronavirus, Explained', 'Exterminadores do Além Contra a Loira do Banheiro', 'Greatest Events of WWII in Colour', 'History 101',
                            'Kubo and the Two Strings', 'Little Witch Academia', 'Ritoru wicchi akademia: Mahoujihikake no parêdo', 'Uchiage hanabi, shita kara miru ka? Yoko kara miru ka?',
                            'The Martian', 'Tonari no Totoro', 'Mission Control: The Unsung Heroes of Apollo', 'NOVA: Black Hole Apocalypse', 'NOVA: Death Dive to Saturn',
                            'The Universe', 'Nakitai watashi wa neko o kaburu', 'One Punch Man: Wanpanman', 'ParaNorman',
                            'The Secret Life of Pets', 'Sangatsu no Lion', 'WWII in Color: Road to Victory', 'Sherlock',
                            'Si shi qing chun', 'Space Force', 'Karakai Jouzu no Takagi-san', 'The Search for Life: The Drake Equation',
                            'The Walking Dead', 'Toradora!', 'Tron: Legacy', 'Full House',
                            'The Fresh Prince of Bel-Air', 'Hoshi o ou kodomo', 'Kimi no na wa.'])
original_titles

array(['Toki o kakeru shôjo', 'Sen to Chihiro no kamikakushi',
       'Koe no katachi',
       'Ano hi mita hana no namae o bokutachi wa mada shiranai',
       'Atlantis: The Lost Empire', 'SpongeBob SquarePants',
       'The SpongeBob Movie: Sponge Out of Water',
       'Challenger: The Final Flight', 'Coraline',
       'Ôkami kodomo no Ame to Yuki', 'Inside Out', 'School of Rock',
       'Coronavirus, Explained',
       'Exterminadores do Além Contra a Loira do Banheiro',
       'Greatest Events of WWII in Colour', 'History 101',
       'Kubo and the Two Strings', 'Little Witch Academia',
       'Ritoru wicchi akademia: Mahoujihikake no parêdo',
       'Uchiage hanabi, shita kara miru ka? Yoko kara miru ka?',
       'The Martian', 'Tonari no Totoro',
       'Mission Control: The Unsung Heroes of Apollo',
       'NOVA: Black Hole Apocalypse', 'NOVA: Death Dive to Saturn',
       'The Universe', 'Nakitai watashi wa neko o kaburu',
       'One Punch Man: Wanpanman', 'ParaNorman',
      

In [28]:
mapped_titles = dict(zip(titles, original_titles))
mapped_titles

{'A Garota que conquistou o tempo': 'Toki o kakeru shôjo',
 'A Viagem de Chihiro': 'Sen to Chihiro no kamikakushi',
 'A Voz do Silêncio – Koe No Katachi': 'Koe no katachi',
 'Ano Hana': 'Ano hi mita hana no namae o bokutachi wa mada shiranai',
 'Atlantis - O reino perdido': 'Atlantis: The Lost Empire',
 'Bob Esponja': 'SpongeBob SquarePants',
 "Bob Esponja - Um Herói Fora D'Água": 'The SpongeBob Movie: Sponge Out of Water',
 'Challenger: Voo Final': 'Challenger: The Final Flight',
 'Coraline e o mundo secreto': 'Coraline',
 'Crianças Lobo': 'Ôkami kodomo no Ame to Yuki',
 'Divertida Mente': 'Inside Out',
 'Escola de rock': 'School of Rock',
 'Explicando... O Coronavírus': 'Coronavirus, Explained',
 'Exterminadores do Além Contra a Loira do Banheiro': 'Exterminadores do Além Contra a Loira do Banheiro',
 'Grandes Momentos da Segunda Guerra em Cores': 'Greatest Events of WWII in Colour',
 'História: Direto ao Assunto': 'History 101',
 'Kubo e as Cordas Mágicas': 'Kubo and the Two Strings

In [29]:
netflix_df['Title'].replace(mapped_titles, inplace=True)
netflix_df.sample(10)

,Title,Date,Episode
99,Toradora!,05/08/2020,1: Tigre de Bolso da Felicidade
266,Sherlock,14/12/2019,4: O detetive mentiroso
72,Sangatsu no Lion,29/10/2020,1: Episódio 13
317,The Fresh Prince of Bel-Air,04/11/2019,4: Uma nova carreira
263,Karakai Jouzu no Takagi-san,16/12/2019,2: Episódio 1
181,Full House,19/01/2020,4: Proposta indecente
176,Full House,20/01/2020,4: Las Joey Vegas
203,Full House,08/01/2020,3: O melhor aniversário do mundo
175,Full House,20/01/2020,4: Dieta secreta
109,Toradora!,03/08/2020,1: Aquela cara que você fez


In [54]:
from IPython.display import clear_output

def get_id_from_imdb(title_name):
  titles = spark.sql(f"SELECT * FROM akas_df WHERE title = '{title_name}' AND types = 'imdbDisplay'").toPandas()
  titles = titles.drop_duplicates(subset=['titleId'])
  titles = titles.reset_index(drop=True)

  """
  ids_size = len(titles.index)

  if ids_size > 1:
    clear_output(wait=True)
    print(f'Escolha um ID (index) para <<{title_name}>>:')
    display(titles)
    input_id = int(input('>> '))
    id = titles['titleId'][titles.index[input_id]]
  elif ids_size == 1:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None
  """

  if len(titles) > 0:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None

  title_id = id if id != None else np.nan
  return title_id

In [55]:
title_ids = [get_id_from_imdb(x) for x in original_titles]
title_ids

['tt0808506',
 'tt0245429',
 'tt5323662',
 nan,
 'tt0230011',
 'tt0206512',
 'tt2279373',
 'tt12930534',
 'tt0327597',
 nan,
 'tt0073174',
 'tt0332379',
 'tt12189310',
 'tt8753660',
 'tt9103932',
 'tt11958648',
 'tt4302938',
 'tt6352180',
 nan,
 nan,
 'tt2910906',
 'tt0096283',
 'tt5959952',
 nan,
 nan,
 'tt1051155',
 nan,
 'tt4508902',
 'tt1623288',
 'tt2709768',
 'tt6074794',
 'tt16477402',
 'tt0337593',
 nan,
 'tt11309924',
 'tt7808344',
 nan,
 'tt0028478',
 'tt1279024',
 'tt1104001',
 'tt0092359',
 'tt0098800',
 nan,
 'tt5311514']

In [58]:
def categorise(row):
  for i in range(len(original_titles)):
    if(row['Title'] == original_titles[i]):
      return title_ids[i]
  return np.nan

In [61]:
netflix_df['titleId'] = netflix_df.apply(lambda row: categorise(row), axis=1)
netflix_df.head()

,Title,Date,Episode,titleId
0,WWII in Color: Road to Victory,14/02/2022,1: Ocupação da Itália,tt16477402
1,WWII in Color: Road to Victory,08/02/2022,1: Batalha de Kursk,tt16477402
2,WWII in Color: Road to Victory,31/01/2022,1: Ocupação do Norte da África,tt16477402
3,WWII in Color: Road to Victory,30/12/2021,1: Batalha do Atlântico,tt16477402
4,WWII in Color: Road to Victory,27/12/2021,1: Dunquerque,tt16477402


In [63]:
netflix_df = netflix_df[['titleId', 'Title', 'Episode', 'Date']]

In [64]:
netflix_df.head()

,titleId,Title,Episode,Date
0,tt16477402,WWII in Color: Road to Victory,1: Ocupação da Itália,14/02/2022
1,tt16477402,WWII in Color: Road to Victory,1: Batalha de Kursk,08/02/2022
2,tt16477402,WWII in Color: Road to Victory,1: Ocupação do Norte da África,31/01/2022
3,tt16477402,WWII in Color: Road to Victory,1: Batalha do Atlântico,30/12/2021
4,tt16477402,WWII in Color: Road to Victory,1: Dunquerque,27/12/2021


In [65]:
from google.colab import files

netflix_df.to_csv('NetflixIMDb.csv', index=False, encoding = 'utf-8') 
files.download('NetflixIMDb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Área de Testes

In [44]:
akas_df.filter(akas_df["title"] == 'Greatest Events of WWII in Colour').show()

+---------+--------+--------------------+------+--------+-----------+----------+---------------+
|  titleId|ordering|               title|region|language|      types|attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+----------+---------------+
|tt9103932|      11|Greatest Events o...|    AU|      \N|imdbDisplay|        \N|              0|
|tt9103932|      13|Greatest Events o...|    IN|      en|imdbDisplay|        \N|              0|
|tt9103932|      14|Greatest Events o...|    \N|      \N|   original|        \N|              1|
|tt9103932|       1|Greatest Events o...|    CA|      en|imdbDisplay|        \N|              0|
|tt9103932|       2|Greatest Events o...|    CA|      fr|imdbDisplay|        \N|              0|
|tt9103932|       4|Greatest Events o...|    US|      \N|imdbDisplay|        \N|              0|
|tt9103932|       6|Greatest Events o...|    GB|      \N|imdbDisplay|        \N|              0|
+---------+--------+----------

In [ ]:
ratings_df.filter(ratings_df["tconst"] == 'tt16477402').show()

+----------+-------------+--------+
|    tconst|averageRating|numVotes|
+----------+-------------+--------+
|tt16477402|          8.2|    1227|
+----------+-------------+--------+



In [ ]:
episode_df.filter(episode_df["parentTconst"] == 'tt16477402').show()

+----------+------------+------------+-------------+
|    tconst|parentTconst|seasonNumber|episodeNumber|
+----------+------------+------------+-------------+
|tt16542412|  tt16477402|           1|            1|
|tt16542414|  tt16477402|           1|            2|
|tt16542592|  tt16477402|           1|            3|
|tt16542830|  tt16477402|           1|            4|
|tt16542940|  tt16477402|           1|            5|
|tt16543116|  tt16477402|           1|            6|
|tt16543212|  tt16477402|           1|            8|
|tt16543224|  tt16477402|           1|            7|
|tt16543454|  tt16477402|           1|            9|
|tt16543590|  tt16477402|           1|           10|
+----------+------------+------------+-------------+



In [125]:
basics_df.filter(basics_df["tconst"] == 'tt6756500').show()

+---------+---------+------------+-------------+-------+---------+-------+--------------+-----------+
|   tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|     genres|
+---------+---------+------------+-------------+-------+---------+-------+--------------+-----------+
|tt6756500|    short|        NOVA|         NOVA|      0|     2015|     \N|            \N|Drama,Short|
+---------+---------+------------+-------------+-------+---------+-------+--------------+-----------+



In [32]:
# Registering a table
akas_df.registerTempTable("akas_df")
spark.sql("SELECT * FROM akas_df").show(3)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|           \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|           \N|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 3 rows



In [147]:
df = spark.sql("SELECT * FROM akas_df WHERE title LIKE 'WWII in Color: Road to Victory%' AND types = 'imdbDisplay'").toPandas()
df

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt16477402,10,WWII in Color: Road to Victory,ID,en,imdbDisplay,\N,0
1,tt16477402,11,WWII in Color: Road to Victory,AE,\N,imdbDisplay,\N,0
2,tt16477402,20,WWII in Color: Road to Victory,IN,hi,imdbDisplay,\N,0
3,tt16477402,3,WWII in Color: Road to Victory,NL,\N,imdbDisplay,\N,0
4,tt16477402,4,WWII in Color: Road to Victory,DE,\N,imdbDisplay,\N,0
5,tt16477402,5,WWII in Color: Road to Victory,PH,en,imdbDisplay,\N,0
6,tt16477402,6,WWII in Color: Road to Victory,GB,\N,imdbDisplay,\N,0
7,tt16477402,7,WWII in Color: Road to Victory,SG,en,imdbDisplay,\N,0
8,tt16477402,8,WWII in Color: Road to Victory,CA,en,imdbDisplay,\N,0
9,tt16477402,9,WWII in Color: Road to Victory,TR,tr,imdbDisplay,\N,0


In [150]:
df.groupby('titleId').count()

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt16477402,10,10,10,10,10,10,10


In [157]:
df.groupby('titleId').count().index[0]

'tt16477402'

In [151]:
len(df.groupby('titleId'))

1

In [ ]:
basics_df.registerTempTable("basics_df")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [ ]:
spark.sql("SELECT * FROM basics_df WHERE primaryTitle LIKE '%Dunkirk%'").toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0051565,movie,Dunkirk,Dunkirk,0,1958,\N,135,"Action,Drama,History"
1,tt0058740,movie,Weekend at Dunkirk,Week-end à Zuydcoote,0,1964,\N,119,"Drama,War"
2,tt0243228,video,Dunkirk: The Battle for France,Dunkirk: The Battle for France,0,1989,\N,60,"Documentary,History,War"
3,tt0394875,tvEpisode,The Dunkirk Dilemma,The Dunkirk Dilemma,0,1997,\N,60,\N
4,tt0398983,tvMovie,Dunkirk: A Human Endeavour,Dunkirk: A Human Endeavour,0,1990,\N,\N,Documentary
...,...,...,...,...,...,...,...,...,...
69,tt9037796,tvEpisode,Dunkirk 1940: the Great Escape,Dunkirk 1940: the Great Escape,0,1990,\N,\N,"Documentary,History"
70,tt9194204,tvEpisode,Dunkirk,Dunkirk,0,2018,\N,28,"Comedy,Documentary,History"
71,tt9339030,tvEpisode,America's Dunkirk Moment,America's Dunkirk Moment,0,2018,\N,\N,News
72,tt9494654,tvEpisode,Dunkirk,Dunkirk,0,2017,\N,\N,"Comedy,Talk-Show"


In [124]:
ratings_df.registerTempTable("ratings_df")
spark.sql("SELECT * FROM ratings_df WHERE tconst = 'tt6756500'").show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+------+-------------+--------+
|tconst|averageRating|numVotes|
+------+-------------+--------+
+------+-------------+--------+



In [ ]:
ratings_df.filter(ratings_df["tconst"] == 'tt12930534').show()

+----------+-------------+--------+
|    tconst|averageRating|numVotes|
+----------+-------------+--------+
|tt12930534|          7.8|    6546|
+----------+-------------+--------+



In [ ]:
spark.sql("SELECT * FROM \
ratings_df r INNER JOIN akas_df a \
  ON r.tconst = a.titleId \
WHERE r.tconst = 'tt12930534' \
").show()

+----------+-------------+--------+----------+--------+--------------------+------+--------+-----------+-----------+---------------+
|    tconst|averageRating|numVotes|   titleId|ordering|               title|region|language|      types| attributes|isOriginalTitle|
+----------+-------------+--------+----------+--------+--------------------+------+--------+-----------+-----------+---------------+
|tt12930534|          7.8|    6546|tt12930534|      10|          Challenger|    AU|      \N|         \N|short title|              0|
|tt12930534|          7.8|    6546|tt12930534|      11|Challenger: Ostat...|    PL|      \N|imdbDisplay|         \N|              0|
|tt12930534|          7.8|    6546|tt12930534|       1|Challenger: Voo F...|    BR|      \N|imdbDisplay|         \N|              0|
|tt12930534|          7.8|    6546|tt12930534|       2|Der letzte Flug d...|    DE|      \N|imdbDisplay|         \N|              0|
|tt12930534|          7.8|    6546|tt12930534|       3|Challenger: Η 

In [ ]:
spark.stop()